$$ y_{in} = F_{act}((Im+n_{in} W_{in}) + bias) $$
$$ Im^* = F_{act}(y_{in} W_{in}^T) + n_{out} $$

$$ Cost = \sqrt{\langle|Im-Im^*|\rangle} + \lambda \langle r \rangle $$

In [1]:
#Dependencies
import os
import numpy as np
import pylab as pl
import tensorflow as tf
import matplotlib.pyplot as plt

import utils.plotutils as plu
import utils.imreadin as imr
#import utils.dirutils as diru

#code to reload
#import imp
#imp.reload(dir)

%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'dpi' : 200} #plotting pretty

In [2]:
class aec_model(object):
    
    def __init__(self, params):
      params = self.add_params(params)
      self.params = params
      self.make_dirs()
      self.graph = self.make_graph()
    
    def add_params(self, params):  
        params['compression'] = params['imxlen']*params['imylen']/params['nneurons']
        params['savefolder'] = str('./image_output/actfun_'+ params['model_type']+
                                   '_hiddenneurons_'+ str(params['nneurons'])+
                                   '_noisein_'+ str(params['noise_x'])+
                                   '_noiseout_'+ str(params['noise_r'])+
                                   '_lambda_'+ str(params['lambd'])+'/')
        return(params)
        
    def make_dirs(self):
        if not os.path.exists(self.params['savefolder']):
            os.makedirs(self.params['savefolder'])
        
    def make_graph(self):
    
        print('Compressing by',self.params['compression'],'for a total of',self.params['nneurons'],'neurons')

        #setup our graph
        #tf.reset_default_graph()
        mygraph = tf.Graph()
        with mygraph.as_default():

            #input images
            with tf.name_scope('input'):
                self.x = tf.placeholder(tf.float32, shape=[self.params["batchsize"], 
                                                           self.params["imxlen"]*self.params["imylen"]])

            #activation function type
            with tf.name_scope('activation_function'):
                self.act_fun = self.params['model_type']

            #noises
            with tf.name_scope('noises'):
                self.noisexsigma = self.params['noise_x']
                self.noisersigma = self.params['noise_r']

            #function to add noise
            with tf.name_scope("add_noise"):
                def add_noise(input_layer, std):
                    noise = tf.random_normal(shape=tf.shape(input_layer), mean=0.0, stddev=std, dtype=tf.float32) 
                    return tf.add(input_layer,noise)

            #weights
            with tf.variable_scope("weights"):
                
                weights_kernel = tf.random_normal([self.params['imxlen']*self.params['imylen'],
                                                         self.params['nneurons']],
                                                        dtype=tf.float32,stddev=0.1)
                
                self.win = tf.get_variable('weights_in',initializer=weights_kernel)
                self.wout = tf.get_variable('weights_out',initializer=tf.transpose(weights_kernel))

            #bias
            with tf.variable_scope("bias"):
                self.bias = tf.Variable(tf.random_normal([self.params['nneurons']],dtype=tf.float32,stddev=0.1))

            #lambda
            with tf.name_scope('lambda'):
                self.lambd = self.params['lambd']

            #learning_rate
            with tf.name_scope('learning_rate'):
                self.learning_rate = self.params['learning_rate']

            #nonlienarities
            with tf.name_scope("nonlienarities"):
                #define nonlinearities
                def tanh_fun(arg):
                    return tf.nn.tanh(arg) 
                def sigmoid_fun(arg):
                    return tf.nn.sigmoid(arg) 
                def relu_fun(arg):
                    return tf.nn.relu(arg) 
                def no_fun(arg):
                    return arg

            #encoding part of model
            with tf.name_scope("encoding"):
                #calculate input
                linearin = tf.add(tf.matmul(add_noise(self.x,self.params['noise_x']),self.win),self.bias) #add noise to input, and multiply by weights
                yin = tf.case({tf.equal(self.act_fun,'tanh'): (lambda: tanh_fun(linearin)),
                               tf.equal(self.act_fun,'sigmoid'): (lambda: sigmoid_fun(linearin)),
                               tf.equal(self.act_fun,'relu'): (lambda: relu_fun(linearin))},
                              default=(lambda: no_fun(linearin)),
                              exclusive=True)
                self.yin = add_noise(yin,self.params['noise_r'])


            #output part of model
            with tf.name_scope("decoding"):
                #calculate output (reconstruction)
                self.xp = tf.matmul(self.yin,self.wout) #add noise to inner layer, and multiply by weight transpose
                #self.xp = tf.case({tf.equal(self.act_fun,'tanh'): (lambda: tanh_fun(linearout)),
                #                    tf.equal(self.act_fun,'sigmoid'): (lambda: sigmoid_fun(linearout)),
                #                    tf.equal(self.act_fun,'relu'): (lambda: relu_fun(linearout))},
                #                    default=(lambda: no_fun(linearout)),
                #                    exclusive=True, name='output_nonlienarity')

            #calculate cost
            with tf.name_scope("cost_function"):
                self.cost = tf.sqrt(tf.reduce_mean(tf.square(self.x-self.xp))) - tf.reduce_sum(tf.abs(self.yin*self.lambd))

            #train our model
            with tf.name_scope("training_step"):
                self.train_step = tf.train.AdamOptimizer(self.learning_rate).minimize(self.cost)

            # create a summary for our cost, im, reconstruction, & weights
            with tf.name_scope('cost_viz'):
                tf.summary.scalar("cost", self.cost)

            with tf.name_scope('image_viz'):    
                x_t = tf.reshape(self.x,(self.params['batchsize'],self.params['imxlen'],self.params['imylen'],1))
                tf.summary.image("image", x_t, max_outputs=self.params["batchsize"])

            with tf.name_scope('recon_viz'):
                xp_t = tf.reshape(self.xp,(self.params['batchsize'],self.params['imxlen'],self.params['imylen'],1))
                tf.summary.image("recon", xp_t,max_outputs=self.params["batchsize"])

            with tf.name_scope('inweights_viz'):    
                inwin_t = tf.reshape(tf.transpose(self.win),
                                   (self.params['nneurons'],
                                    self.params['imxlen'],
                                    self.params['imylen'],1))
                tf.summary.image("inweights", inwin_t, max_outputs=self.params['nneurons'])
                
            with tf.name_scope('outweights_viz'):    
                outwin_t = tf.reshape(self.win,
                                   (self.params['nneurons'],
                                    self.params['imxlen'],
                                    self.params['imylen'],1))
                tf.summary.image("outweights", outwin_t, max_outputs=self.params['nneurons'])

            # merge all summaries into a single "operation" which we can execute in a session 
            self.summary_op = tf.summary.merge_all()

        return(mygraph)


In [3]:
#make session and train model
def train_model(aec):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(graph = aec.graph) as sess:

        #initialize vars
        init = tf.global_variables_initializer()
        sess.run(init)

        #summary writer for tensorboard
        writer = tf.summary.FileWriter(aec.params['savefolder'],
                                       graph=tf.get_default_graph())

        #save evolution of system over training
        cost_evolution = []
        wmean_evolution = []

        inweights_evolution = []
        outweights_evolution = []
        
        images = []
        recons = []
        print('{} hidden neurons, noise_in at {}, noise_out at {}, lambda at {}'.format(aec.params['nneurons'],
                                                                                          aec.params['noise_x'],
                                                                                          aec.params['noise_r'],
                                                                                          aec.params['lambd']))
        
        print('Training {} iterations in {} epochs... '.format(aec.params['iterations'],
                                                                   aec.params['epochs']))
        for epoch in range(aec.params['epochs']):
            #print('Epoch {}: '.format(epoch+1))
            np.random.shuffle(vhimgs.images)
            for ii in range(aec.params['iterations']):

                #reshape our images for feeding to dict
                image = np.reshape(vhimgs.images[ii*aec.params['batchsize']:(1+ii)*aec.params['batchsize'],:,:],
                                   (aec.params['batchsize'],
                                    aec.params['imxlen']*aec.params['imylen'])).astype(np.float32)

                #setup params to send to dictionary
                feeddict = {aec.x: image}
    #                        aec.params['model_type']: params['model_type'],
    #                        aec.params['noise_x']: params['noise_x'],
    #                        aec.params['noise_r']: params['noise_r'],
    #                        aec.params['lambd']: params['lambd'],
    #                        aec.params['learning_rate']: params['learning_rate']
    #                        }

                #run our session
                sess.run(aec.train_step, feed_dict=feeddict)

                #save evolution of params
                objcost, inws = sess.run([aec.cost, aec.win], feed_dict=feeddict)
                cost_evolution.append(objcost)
                wmean_evolution.append(np.mean(inws))

                #save detailed parameters 10 times over the total evolution
                if(ii%(int((aec.params['iterations']*aec.params['epochs'])/10))==0):
                    #print(str(ii)+', ',end="")
                    #dump our params
                    win, wout, img, recon = sess.run([aec.win, aec.wout, aec.x, aec.xp], feed_dict=feeddict)
                    #save our weights, image, and reconstruction
                    inweights_evolution.append(win)
                    outweights_evolution.append(wout)
                    imshape = [aec.params['batchsize'],
                              aec.params['imxlen'],
                              aec.params['imylen']]

                    images.append(np.reshape(img, imshape))
                    recons.append(np.reshape(recon, imshape))

        #summarize final params
        summary, objcost, inws, outws = sess.run([aec.summary_op, aec.cost, aec.win, aec.wout], feed_dict=feeddict)
        cost_evolution.append(objcost)
        wmean_evolution.append(np.mean(inws))
        final_inweights = aec.win
        final_outweights = aec.wout
        writer.add_summary(summary,ii)
        writer.close()

    print('Done!')
    
    return(cost_evolution,
            wmean_evolution,
            inweights_evolution,
            outweights_evolution,
            images,
            recons,
            final_inweights,
            final_outweights)

In [4]:
#set parameters for parameter sweep
params = {} #make a dictionary

#parameters constant for all
params["patchsize"] = 16
params["imxlen"] = params["patchsize"]
params["imylen"] = params["patchsize"]
params["iterations"] = 40000
params["epochs"] = 5

params["batchsize"] = 100
params["learning_rate"] = 0.01

#params for sweeping
nneurons = [25,64,100]
lambdas = [0,1e-6,1e-5,1e-4]
models = ['sigmoid','tanh','relu']
#batchsizes = [100,1000]
#learning_rates = [0.05,0.01,0.1]
noise_xs = [1,1e-1,1e-3,1e-5,0]
noise_rs = [1,1e-1,1e-3,1e-5,0]

In [5]:
vhimgs, params['nimages'] = imr.check_n_load_ims(params['patchsize'], params['iterations'])

Loading Van Hateren Natural Image Database...
normalizing full images...
sectioning into patches....
Done Loading!
Done Shuffling!
Images Loaded.


In [6]:
for neurons in nneurons:
    params['nneurons'] = neurons
    for model_type in models:
        params['model_type'] = model_type      
        for lambd in lambdas:
            params['lambd'] = lambd
            for nx in noise_xs:
                params['noise_x'] = nx
                for nr in noise_rs:
                    params['noise_r'] = nr

                    #make our model
                    aec = aec_model(params)
                    #train it
                    [cost_evolution,
                     wmean_evolution,
                     inweights_evolution,
                     outweights_evolution,
                     images,
                     recons,
                     final_inweights,
                     final_outweights] = train_model(aec)

                    plu.save_plots(aec,
                                    cost_evolution,
                                    wmean_evolution,
                                    inweights_evolution,
                                    outweights_evolution,
                                    images,
                                    recons,
                                    final_inweights,
                                    final_outweights)


Compressing by 10.24 for a total of 25 neurons
25 hidden neurons, noise_in at 1, noise_out at 1, lambda at 0
Training 40000 iterations in 5 epochs... 
Done!
Compressing by 10.24 for a total of 25 neurons
25 hidden neurons, noise_in at 1, noise_out at 0.1, lambda at 0
Training 40000 iterations in 5 epochs... 
Done!
Compressing by 10.24 for a total of 25 neurons
25 hidden neurons, noise_in at 1, noise_out at 0.001, lambda at 0
Training 40000 iterations in 5 epochs... 
Done!
Compressing by 10.24 for a total of 25 neurons
25 hidden neurons, noise_in at 1, noise_out at 1e-05, lambda at 0
Training 40000 iterations in 5 epochs... 
Done!
Compressing by 10.24 for a total of 25 neurons
25 hidden neurons, noise_in at 1, noise_out at 0, lambda at 0
Training 40000 iterations in 5 epochs... 
Done!
Compressing by 10.24 for a total of 25 neurons
25 hidden neurons, noise_in at 0.1, noise_out at 1, lambda at 0
Training 40000 iterations in 5 epochs... 
Done!
Compressing by 10.24 for a total of 25 neurons

KeyboardInterrupt: 